# PII Detection Modes with a Custom Test

This notebook shows how to initialize ValidMind, implement a custom test that emits PII, and observe behavior differences under each `VALIDMIND_PII_DETECTION` mode when running the test with `validmind.tests.run_test`.


## Prerequisites

- `validmind` installed with PII extras:

```bash
%pip install -q validmind[pii-detection]
```

- A ValidMind model registered. We'll initialize the library using your model snippet.


In [ ]:
%pip install -q "validmind[pii-detection]"

## Initialize ValidMind

Initialize using your model code snippet or a `.env` file, as shown in other quickstarts.


In [ ]:
# Load your model identifier credentials from an `.env` file
%load_ext dotenv
%dotenv .env

# Or initialize with your code snippet
import validmind as vm

vm.init(
    # api_host="...",
    # api_key="...",
    # api_secret="...",
    # model="...",
)

## Create a custom test that emits PII

We'll create a custom test that returns:
- A description string containing PII (name, email, phone)
- A small table containing PII in columns

This mirrors the structure used in other custom test notebooks and will exercise both table and description PII detection paths.


In [ ]:
import pandas as pd

from validmind import test

@test("my_pii_demo.PIIEmittingTest")
def pii_emitting_test():
    """A demo test that returns PII"""
    return pd.DataFrame(
        {
            "name": ["Jane Smith", "John Doe", "Alice Johnson"],
            "email": [
                "jane.smith@bank.example",
                "john.doe@company.example",
                "alice.johnson@service.example",
            ],
            "phone": ["(212) 555-9876", "(415) 555-1234", "(646) 555-5678"],
        }
    )

## Run the test under different PII detection modes

We'll switch `VALIDMIND_PII_DETECTION` across modes and run the same test with `validmind.tests.run_test`. We catch exceptions to observe blocking behavior.


In [ ]:
import os
from validmind.tests import run_test

MODES = ["disabled", "test_results", "test_descriptions", "all"]

for mode in MODES:
    print("\n=== Mode:", mode, "===")
    os.environ["VALIDMIND_PII_DETECTION"] = mode
    try:
        result = run_test("my_pii_demo.PIIEmittingTest")

        # check if the description was generated
        if not result._was_description_generated:
            print("Blocked: Test Description Generation was not run due to PII")
        else:
            print("Description was generated by LLM")

        # Try logging (this triggers PII checks before upload)
        result.log()
        print("Logging to API succeeded")
    except Exception as e:
        print("Blocked: Test Result was not logged due to PII")
        # print(e)

### Expected behavior by mode

- disabled: No PII checks.
- test_results: Description is generated but result is not logged.
- test_descriptions: Description generation is blocked but result is logged.
- all: Description generation and logging are both blocked.


## Notes

- If you see warnings that Presidio is unavailable, ensure you installed extras: `validmind[pii-detection]`.
- You can override blocking by passing `unsafe=True` to `result.log(unsafe=True)`, but this is not recommended outside controlled workflows.
